In [ ]:
import pynbody
import tangos
import edge_tangos_properties as etp
import numpy as np
import statistics

In [ ]:
tangos.core.init_db('/scratch/dp191/shared/tangos/HALO_DATABASE_HERE')
# halo tangos database = Halo339.db, Halo261.db
sim = tangos.get_simulation('HALO_SIM_HERE')
# halo sim = Halo339_RT, Halo261_RT, Halo339_wdm3_RT, Halo339_wdm6_RT

In [ ]:
contam, no_stars, mdm, mstar, tangosid, pynbodyid, shrinkcenter, r200c = sim.timesteps[-1].calculate_all('contamination_fraction<0.01', 'M200c_stars>0', 'M200c_DM', 'M200c_stars', 'halo_number()', 'finder_id()', 'shrink_center','r200c')

In [ ]:
# Halo339 - last timestep 00139
# Halo261 - last timestep 00177
# Halo339 wdm 3 - last timestep 00130
# Halo339 wdm 6 - last timestep 00137

In [ ]:
s = pynbody.load('/scratch/dp191/shared/EDGE2_simulations/HALO_SIM_HERE/output_00000/')
s.physical_units()

index=[]
tangosnumber=[]
pynbodynumber=[]
stellar_mass=[]
logstellar_mass=[]
halo_mass=[]
vmags=[]
meanfeh=[]
fehspread=[]
sigmalineofsight=[]
rhalf_pc=[]
rhalf_kpc=[]
agedispersion_gyr=[]
agedispersion_myr=[]
agedifference_gyr=[]
agedifference_myr=[]
numstars=[]
formationtimes_gyr=[]
#dynamicalmass=[]
smplushm=[]
massratio=[]

for i in range(len(contam)):
    if  contam[i] * no_stars[i]:
        s['pos'] -=shrinkcenter[i]
        h = s[pynbody.filt.Sphere(pynbody.array.SimArray(r200c[i], units='kpc'))]
        h.s['tform']
        starparticles = len(h.s)
        deltaage = np.ptp(h.s['tform']) 
        sigmaage = np.std(h.s['tform'])
        formtimes_gyr = statistics.median(h.s['tform']) 
        halo = sim.timesteps.halos[tangosid[i]-1]
# in case etp is needed for calculating v band mags and velocity dispersion
    #mag_calculator = etp.stars.Magnitude(h)
    #v_band, u_band, b_band, r_band = mag_calculator.calculate(h, sim.timesteps[-1].halos[tangosid[i]-1])
    #stellar1Dveldisp = etp.stars.VelocityDispersion(h)
    #sigma_3d, sigma_1d = stellar1Dveldisp.calculate(h, sim.timesteps[-1].halos[tangosid[i]-1])
        rhalf = pynbody.analysis.luminosity.half_light_r(h,cylindrical=True)
        metals = etp.stars.AbundanceRatios(h)
        try:
            iron_hydrogen_ratios, oxygen_iron_ratios, mean_iron_hydrogen_ratio = metals.calculate(h, sim.timesteps[-1].halos[tangosid[i]-1])
        except: ZeroDivisisonError:
            iron_hydrogen_ratios = np.nan
            mean_iron_hydrogen_ratio = np.nan
        ironhydrogenspread = np.std([iron_hydrogen_ratios])
        v_band = halo['stellar_V_mag']
# these three properties have not been calculated for all halos in new databases
    #rhalf = halo['stellar_projected_halflight'] 
    #mean_iron_hydrogen_ratio = halo['mean_iron_hydrogen_ratio']
    #ironhydrogenspread = np.std(halo['iron_hydrogen_ratios'])
        stellar1Dveldisp = halo['stellar_1D_velocity_dispersion']
        s['pos'] += shrinkcenter[i]
        print(i, tangosid[i], pynbodyid[i])
        #all properties
        index+=[i]
        tangosnumber+=[tangosid[i]]
        pynbodynumber+=[pynbodyid[i]]
        stellar_mass+=[mstar[i]]
        logstellar_mass+=[np.log10(mstar[i])]
        halo_mass+=[mdm[i]]
        vmags+=[v_band]
        meanfeh+=[mean_iron_hydrogen_ratio]
        fehspread+=[ironhydrogenspread]
        sigmalineofsight+=[stellar1Dveldisp]
        rhalf_kpc+=[rhalf]
        rhalf_pc+=[rhalf*1000]
        agedispersion_gyr+=[sigmaage]
        agedispersion_myr+=[sigmaage*1000]
        agedifference_gyr+=[deltaage]
        agedifference_myr+=[deltaage*1000]
        numstars+=[starparticles]
        formationtimes_gyr+=[formtimes_gyr]
        #dynamicalmass+=[np.log10(((rhalf*1000)*(stellar1Dveldisp**2))/(0.004301))]
        smplushm+=[mstar[i]+mdm[i]]
        massratio+=[np.log10(mstar[i]/(mstar[i]+mdm[i]))]
print('total number of halos',len(index))
np.savetxt('titlehere.csv',np.transpose(np.array([index,tangosnumber,pynbodynumber,stellar_mass,logstellar_mass,halo_mass,vmags,meanfeh,fehspread,sigmalineofsight,rhalf_kpc,rhalf_pc,agedispersion_gyr,agedispersion_myr,agedifference_myr,numstars,formationtimes_gyr,smplushm,massratio])),header='Index,Tangos Halo ID,Pynbody Halo ID,Stellar Mass,Log Stellar Mass,Halo Mass,V-band Mags,Mean FeH,FeH Spread,Sigma Line of Sight,Half Light Radius (kpc),Half Light Radius (pc),Age Dispersion (Gyr),Age Dispersion (Myr),Age Difference (Myr),Number of Stars,Formation Time (Gyr),Stellar Mass Plus Halo Mass,Mass Ratio',delimiter=',',comments='',fmt='%s')

